# Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Firdaus Arif Ramadhani
- **Email:** firdausarief65@gmail.com
- **ID Dicoding:** FIRDAUS ARIF RAMADHANI

## Menentukan Pertanyaan Bisnis

- 

## Import Semua Packages/Library yang Digunakan

In [2773]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2774]:
from deep_translator import GoogleTranslator

text_portuguese = "Olá, como você está?"
translated_text = GoogleTranslator(source="portuguese", target="english").translate(
    text_portuguese
)

print(translated_text)

Hello, how are you?


## Data Wrangling

### Gathering Data

#### Data Tabel `products_df`

In [2775]:
products_df = pd.read_csv("e-commerce_public_dataset/products_dataset.csv")
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


**Insight:**\
Dataset products_df berisi informasi terkait produk.\
Informasi yang tersedia diantaranya:
1. `product_id`: ID unik untuk setiap produk.
2. `product_category_name`: Panjang nama produk dalam karakter.
3. `product_name_lenght`: Panjang deskripsi produk.
4. `product_photos_qty`: Jumlah foto yang tersedia untuk produk.
5. `product_weight_g`: Berat produk.
6. `product_length_cm`, `product_height_cm`, dan `product_width_cm`: DImensi produk.

#### Data Tabel `product_category_translation_df`

In [2776]:
product_category_translation_df = pd.read_csv(
    "e-commerce_public_dataset/product_category_name_translation.csv"
)
product_category_translation_df.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


**Insight:**\
Dataset `product_category_translation_df` memuat terjemahan nama kategori produk dari bahasa Portugis ke bahasa Inggris.\
Informasi yang tertera diantaranya:
1. `product_category_name`: Nama kategori dalam bahasa Portugis.
2. `product_category_name_english`: Nama kategori dalam bahasa Inggris.

#### Data Tabel `order_reviews_df`

In [2777]:
order_reviews_df = pd.read_csv("e-commerce_public_dataset/order_reviews_dataset.csv")
order_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01 00:00:00,2018-03-02 10:26:53


**Insight:**\
Dataset `order_reviews_df` berisi informasi mengenai ulasan pelanggan terhadap pesanan yang telah mereka terima.\
Informasi yang tertera diantaranya:
1. `review_id`: ID unik untuk setiap ulasan.
2. `order_id`: ID unik dari pesanan.
3. `review_score`: Skor ulasan yang diberikan oleh pelanggan, pada skala 1-5.
4. `review_comment_title` & `review_comment_message`: Komentar pelanggan.
5. `review_creation_date` $ `review_answer_timestamp`: Waktu ketika ulasan dibuat dan dijawab.

#### Data Tabel `order_payments`

In [2778]:
order_payments_df = pd.read_csv("e-commerce_public_dataset/order_payments_dataset.csv")
order_payments_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


**Insight:**\
Dataset `order_payments` ini berisi informasi mengenai pembayaran yang dilakukan pelanggan untuk pesanan yang mereka buat.\
Informasi yang tertera diantaranya:
1. `order_id`: ID unik untuk setiap pesanan.
2. `payment_sequential`: Urutan pembayaran untuk setiap pesanan.
3. `payment_type`: Jenis pembayaran yang digunakan oleh pelanggan.
4. `payment_installments`: Jumlah cicilan yang diambil oleh pelanggan untuk membayar pesanan.
5. `payment_value`: Nilai total dari setiap pembayaran.

#### Data Tabel `order_items_df`

In [2779]:
order_items_df = pd.read_csv("e-commerce_public_dataset/order_items_dataset.csv")
order_items_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


**Insight:**\
Dataset `order_items_df` berisi informasi mengenai item yang dipesan dalam setiap pesanan.\
Informasi yang tertera diantaranya:
1. `order_id`: ID unik untuk setiap pesanan.
2. `order_item_id`: ID untuk setiap item pesanan dalam pesanan yang sama.
3. `product_id`: ID produk yang dipesan.
4. `seller_id`: ID unik penjual.
5. `shipping_limit_date`: Batas waktu pengiriman item oleh penjual.
6. `price`: Harga jual produk yang dipesan.
7. `freight_value`: Biaya pengiriman item.

#### Data Tabel `geolocation_df`

In [2780]:
geolocation_df = pd.read_csv("e-commerce_public_dataset/geolocation_dataset.csv")
geolocation_df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


**Insight:**\
Dataset `geolocation_df` berisi informasi geolokasi.\
Informasi yang tertera diantaranya:
1. `geolocation_zip_code_prefix`: Kode pos.
2. `geolocation_lat`: Koordinat geografis lintang.
3. `geolocation_lng`: Koordinat geografis bujur.
4. `geolocation_city`: Nama kota.
5. `geolocation_state`: Nama negara bagian.

#### Data Tabel `customers_df`

In [2781]:
customers_df = pd.read_csv("e-commerce_public_dataset/customers_dataset.csv")
customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


**Insight:**\
Dataset `customers_df` berisi informasi mengenai pelanggan yang melakukan pemesanan.\
Informasi yang tertera diantaranya:
1. `customer_id`: ID pelanggan untuk setiap pesanan.
2. `customer_unique_id`: ID unik pelanggan.
3. `customer_zip_code_prefix`: Kode pos pelanggan.
4. `customer_city`: Nama kota pelanggan.
5. `customer_state`: Nama negara bagian pelanggan.

#### Data Tabel `sellers_df`

In [2782]:
sellers_df = pd.read_csv("e-commerce_public_dataset/sellers_dataset.csv")
sellers_df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


**Insight:**\
Dataset `sellers_df` berisi informasi penjual\
Informasi yang tertera diantaranya:
1. `seller_id`: ID penjual.
2. `seller_zip_code_prefix`: Kode pos penjual.
3. `seller_city`: Nama kota penjual.
4. `seller_state`: Nama negara bagian penjual.

#### Data Tabel `orders_df`

In [2783]:
orders_df = pd.read_csv("e-commerce_public_dataset/orders_dataset.csv")
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


**Insight:**\
Dataset `orders_df` berisi informasi mengenai pesanan yang dibuat oleh pelanggan.\
Informasi yang tertera diantaranya:
1. `order_id`: ID unik untuk setiap pesanan.
2. `customer_id`: ID pelanggan yang terkait dengan pesanan.
3. `order_status`: Status pesanan yang menunjukkan tahapan pesanan seperti delivered, shipped, canceled, dll.
4. `order_purchase_timestamp`: Tanggal dan waktu pesanan dibuat.
5. `order_approved_at`: Waktu persetujuan pesanan.
6. `order_delivered_carrier_date`: Tanggal pesanan dikirim ke pelanggan oleh kurir.
7. `order_delivered_customer_date`: Tanggal pesanan diterima oleh pelanggan.
8. `order_estimated_delivery_date`: Estimasi tanggal pengiriman pesanan.

### Assessing Data

#### Menilai Tabel `products_df`

In [2784]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [2785]:
print("\nMissing values in Products dataset:")
products_df.isna().sum()


Missing values in Products dataset:


product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [2786]:
print("Duplicates in Products dataset:", products_df.duplicated().sum())

Duplicates in Products dataset: 0


In [2787]:
products_df["product_weight_g"].describe().round(2)

count    32949.00
mean      2276.47
std       4282.04
min          0.00
25%        300.00
50%        700.00
75%       1900.00
max      40425.00
Name: product_weight_g, dtype: float64

**Insight:**\
Tidak ada data duplikat.\
Terdapat beberapa kolom memiliki nilai yang hilang.\
Rentang nilai `product_weight_g` cukup besar, mulai dari **0.00** hingga **40425** gram, yang menunjukkan adanya anomali pada nilai minimum. Nilai **0.00** pada berat produk tampak tidak logis.

#### Menilai Tabel `product_category_translation_df`

In [2788]:
product_category_translation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [2789]:
print("\nMissing values in Product Category Translation dataset:")
product_category_translation_df.isna().sum()


Missing values in Product Category Translation dataset:


product_category_name            0
product_category_name_english    0
dtype: int64

In [2790]:
print(
    "Duplicates in Product Category Translation dataset:",
    product_category_translation_df.duplicated().sum(),
)

Duplicates in Product Category Translation dataset: 0


**Insight:**\
Setiap kategori produk dalam bahasa Portugis dan terjemahan bahasa Inggris, sesuai dengan jumlah total entri **(71)**.\
Tidak memiliki nilai yang hilang atau duplikat.

#### Menilai Tabel `order_reviews_df`

In [2791]:
order_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [2792]:
print("\nMissing values in Order Reviews dataset:")
order_reviews_df.isna().sum()


Missing values in Order Reviews dataset:


review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [2793]:
print("Duplicates in Order Reviews dataset:", order_reviews_df.duplicated().sum())

Duplicates in Order Reviews dataset: 0


**Insight:**\
Anomali pada kolom `review_creation_date` & `review_answer_timestamp` yang bertipe data *object* (teks).
Terdapat banyak nilai yang hilang pada entri `review_comment_title` dan `review_comment_message`.\
Tidak ada baris yang duplikat.\

#### Menilai Tabel `order_payments_df`

In [2794]:
order_payments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [2795]:
print("\nMissing values in Order Payments dataset:")
order_payments_df.isna().sum()


Missing values in Order Payments dataset:


order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [2796]:
print("Duplicates in Order Payments dataset:", order_payments_df.duplicated().sum())

Duplicates in Order Payments dataset: 0


In [2797]:
order_payments_df.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


**Insight:**\
Tidak ada *missing values*.\
Tidak ada data duplikat.\
Pada kolom `payment_installments` dengan minimum value **0.00**, yang mungkin menunjukkan pembayaran tanpa cicilan.\
Pada kolom `payment_value`, nilai minimum adalah **0.00**, yang mungkin barang tersebut digratiskan atau sedang promo.

#### Menilai Tabel `order_items_df`

In [2798]:
order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [2799]:
print("\nMissing values in Order Items dataset:")
order_items_df.isna().sum()


Missing values in Order Items dataset:


order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [2800]:
print("Duplicates in Order Items dataset:", order_items_df.duplicated().sum())

Duplicates in Order Items dataset: 0


**Insight:**\
Anomali pada kolom `shipping_limit_date` yang bertipe data *object* (teks).\
Tidak ada *missing values*.\
Tidak ada baris yang duplikat.

#### Menilai Tabel `geolocation_df`

In [2801]:
geolocation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [2802]:
print("\nMissing values in Geolocation dataset:")
geolocation_df.isna().sum()


Missing values in Geolocation dataset:


geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [2803]:
print("Duplicates in Geolocation dataset:", geolocation_df.duplicated().sum())

Duplicates in Geolocation dataset: 261831


**Insight:**\
Tidak ada *missing values*.\
Terdapat **261831** baris duplikat, sekitar **26%** dari keseluruhan data.

#### Menilai Tabel `customers_df`

In [2804]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [2805]:
print("\nMissing values in Customers dataset:")
customers_df.isna().sum()


Missing values in Customers dataset:


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [2806]:
print("Duplicates in Customers dataset:", customers_df.duplicated().sum())

Duplicates in Customers dataset: 0


**Insight:**\
Tidak ada anomali tipe data.\
Tidak ada *missing values*.\
Tidak ada baris yang duplikat.

#### Menilai Tabel `sellers_df`

In [2807]:
sellers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [2808]:
print("\nMissing values in Sellers dataset:")
sellers_df.isna().sum()


Missing values in Sellers dataset:


seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [2809]:
print("Duplicates in Sellers dataset:", sellers_df.duplicated().sum())

Duplicates in Sellers dataset: 0


**Insight:**\
Tidak ada anomali tipe data.\
Tidak ada *missing values.*\
Tidak ada bairs yang duplikat.

#### Menilai Tabel `orders_df`

In [2810]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [2811]:
print("Duplicates in Orders dataset:", orders_df.duplicated().sum())

Duplicates in Orders dataset: 0


In [2812]:
print("\nMissing values in Orders dataset:")
orders_df.isna().sum()


Missing values in Orders dataset:


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

**Insight:**\
Anomali pada kolom `order_purchase_timestamp`, `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, dan `order_estimated_delivery_date` dengan tipe data *object* (teks).\
Tidak ada baris yang duplikat.\
Nilai-nilai yang hilang pada dataset berkaitan dengan tahapan proses pesanan, yang bisa disebabkan oleh pesanan yang belum selesai atau pembatalan.\

### Cleaning Data

#### Membersihkan Tabel `products_df`

In [2813]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [2814]:
print("\nMissing values in Products dataset:")
products_df.isna().sum()


Missing values in Products dataset:


product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

<div class="alert alert-block alert-info"><b>Fixing: </b> Missing Values</div> 

In [2815]:
missing_values = products_df[products_df.isna().any(axis=1)]
missing_values.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,NaN,300.0,35.0,7.0,12.0


In [2816]:
# Fill missing values in 'product_category_name' with 'unknown'
products_df.loc[:, "product_category_name"] = products_df[
    "product_category_name"
].fillna("unknown")

In [2817]:
unknown_count = products_df[products_df["product_category_name"] == "unknown"].shape[0]
print(
    f"Number of rows with 'unknown' entries in 'product_category_name': {unknown_count}"
)

Number of rows with 'unknown' entries in 'product_category_name': 610


In [2818]:
print("Product weight statistics:\n")
products_df["product_weight_g"].describe().round(2)

Product weight statistics:



count    32949.00
mean      2276.47
std       4282.04
min          0.00
25%        300.00
50%        700.00
75%       1900.00
max      40425.00
Name: product_weight_g, dtype: float64

In [2819]:
# Product weight with value 0
products_df.loc[
    products_df["product_weight_g"] == 0,
    ["product_id", "product_category_name", "product_weight_g"],
]

,product_id,product_category_name,product_weight_g
9769,81781c0fed9fe1ad6e8c81fca1e1cb08,cama_mesa_banho,0.0
13683,8038040ee2a71048d4bdbbdc985b69ab,cama_mesa_banho,0.0
14997,36ba42dd187055e1fbe943b2d11430ca,cama_mesa_banho,0.0
32079,e673e90efa65a5409ff4196c038bb5af,cama_mesa_banho,0.0


In [2820]:
cama_mesa_banho_df = products_df[
    products_df["product_category_name"] == "cama_mesa_banho"
]

# Replace the value 0 in the 'product_weight_g' column for the cama_mesa_banho cateogry with its median
products_df.loc[
    (products_df["product_category_name"] == "cama_mesa_banho")
    & (products_df["product_weight_g"] == 0),
    "product_weight_g",
] = cama_mesa_banho_df["product_weight_g"].median()

In [2821]:
print("Product weight statistics:\n")
products_df["product_weight_g"].describe().round(2)

Product weight statistics:



count    32949.00
mean      2276.62
std       4281.98
min          2.00
25%        300.00
50%        700.00
75%       1900.00
max      40425.00
Name: product_weight_g, dtype: float64

In [2822]:
# Fill missing values in `product_name_length`, `product_description_length`, and `product_photos_qty` with their mean values
products_df["product_name_lenght"] = products_df["product_name_lenght"].fillna(
    products_df["product_name_lenght"].mean().round(2)
)

products_df["product_description_lenght"] = products_df[
    "product_description_lenght"
].fillna(products_df["product_description_lenght"].mean().round(2))

products_df["product_photos_qty"] = products_df["product_photos_qty"].fillna(
    products_df["product_photos_qty"].mean().round(2)
)

In [2823]:
print("\nMissing values in Products dataset:")
products_df.isna().sum()


Missing values in Products dataset:


product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              2
product_length_cm             2
product_height_cm             2
product_width_cm              2
dtype: int64

In [2824]:
missing_rows = products_df[
    products_df[
        [
            "product_weight_g",
            "product_name_lenght",
            "product_height_cm",
            "product_width_cm",
        ]
    ]
    .isna()
    .any(axis=1)
]

missing_rows

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.00,865.0,3.00,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,unknown,48.48,771.5,2.19,NaN,NaN,NaN,NaN


In [2825]:
columns_to_fill = [
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm",
]

# Specify the category for which you want to fill the NaN value
categories = ["bebes", "unknown"]

# Process for each category
for category in categories:
    for column in columns_to_fill:
        products_df.loc[
            products_df["product_category_name"] == category, column
        ] = products_df[products_df["product_category_name"] == category][
            column
        ].fillna(
            products_df[products_df["product_category_name"] == category][column]
            .mean()
            .round(2)
        )

In [2826]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32951 non-null  object 
 2   product_name_lenght         32951 non-null  float64
 3   product_description_lenght  32951 non-null  float64
 4   product_photos_qty          32951 non-null  float64
 5   product_weight_g            32951 non-null  float64
 6   product_length_cm           32951 non-null  float64
 7   product_height_cm           32951 non-null  float64
 8   product_width_cm            32951 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


**Insigth:**\
*Missing values* sebanyak **610** pada `product_category_name` berhasil diisi dengan nama kategori produk 'unknown' karena hilangnya informasi pada dataset.\
*Missing values* sebanyak **610** pada [`product_name_length`, `product_description_length`, `product_photos_qty`] berhasil diisi dengan nilai *mean*  dari masing-masing kolom mereka.

Nilai *NaN* di kolom `product_weight_g`, `product_length_cm`, `product_height_cm`, dan `product_width_cm` untuk kategori 'bebes' dan 'unknown' telah diisi menggunakan rata-rata (mean) masing-masing kolom untuk setiap kategori. Pendekatan ini memastikan bahwa setiap kategori produk memiliki nilai yang lebih representatif dibanding menggunakan satu nilai median atau mean secara keseluruhan.

Rentang berat produk `product_weight_g` yang dimulai dari **0.00** hingga **40425** gram, menunjukkan adanya anomali dengan adanya berat produk sebesar **0.00** gram yang telah digantikan dengan nilai *median*.

#### Membersihkan Tabel `order_reviews.df`

In [2827]:
order_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


<div class="alert alert-block alert-info"><b>Fixing: </b> Data Types</div> 

In [2828]:
order_reviews_df[["review_creation_date", "review_answer_timestamp"]].dtypes

review_creation_date       object
review_answer_timestamp    object
dtype: object

In [2829]:
order_reviews_df["review_creation_date"] = pd.to_datetime(
    order_reviews_df["review_creation_date"]
)
order_reviews_df["review_answer_timestamp"] = pd.to_datetime(
    order_reviews_df["review_answer_timestamp"]
)

order_reviews_df[["review_creation_date", "review_answer_timestamp"]].dtypes

review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

**Insight:**\
Kolom `review_creation_date` dan `review_answer_timestamp` telah dikonversi menjadi *datetime*.

<div class="alert alert-block alert-info"><b>Fixing: </b> Missing Values</div> 

In [2830]:
print("\nMissing values in Order Reviews dataset:\n")
print(order_reviews_df.isna().sum())


Missing values in Order Reviews dataset:

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


In [2831]:
# Fill the NaN values in review_comment_title and review_comment_message with 'empty'
order_reviews_df.loc[:, ["review_comment_title", "review_comment_message"]] = (
    order_reviews_df.loc[:, ["review_comment_title", "review_comment_message"]].fillna(
        "empty"
    )
)

# Display sample data that contains 'empty'
empty_samples = order_reviews_df[
    (order_reviews_df["review_comment_title"] == "empty")
    | (order_reviews_df["review_comment_message"] == "empty")
]

empty_samples.sample(5)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
97359,8edd0fafd93d51e034ba3c9e68552cf2,9f0c24c74a6da307a862a27b63886f1c,5,empty,Tudo dentro do previsto.,2018-01-19,2018-02-14 13:15:53
66840,6cddbe204d2cfe1f9d39d11cceb2506b,f833a0260cb98fa615a517d4e4a8cdf3,5,empty,Excelente aquisicao,2017-05-11,2017-05-13 03:46:08
40111,44f9c0b307a9ba9bac7a10443ce25b97,ed4702d5c64be8a78291b25080218bea,4,empty,empty,2018-05-17,2018-05-21 12:03:27
12097,5d19d9c423175b2a110a222c845bac83,9039bf11d9ef8467d23a4cfbc162653e,5,empty,empty,2017-11-07,2017-11-07 14:55:22
65109,365f4eada207645c155f471623b7e084,f8207caf0d0b8d07ebb64c397a72793a,5,empty,empty,2017-07-23,2017-07-23 19:42:41


In [2832]:
empty_samples_title = order_reviews_df[
    order_reviews_df["review_comment_title"] == "empty"
]
empty_samples_message = order_reviews_df[
    order_reviews_df["review_comment_message"] == "empty"
]

print(
    f"'empty' values in review_comment_title or review_comment_message: {empty_samples_title.shape[0]}"
)
print(f"'empty' values in review_comment_message: {empty_samples_message.shape[0]}")

'empty' values in review_comment_title or review_comment_message: 87656
'empty' values in review_comment_message: 58247


In [2833]:
print("\nMissing values in Order Reviews dataset:\n")
print(order_reviews_df.isna().sum())


Missing values in Order Reviews dataset:

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


**Insight:**\
*Missing values* pada kolom `review_comment_title` dan `review_comment_message` telah diisi dengan 'empty' untuk mengisi kekosongan data.

#### Membersihkan Tabel `order_items_df`

<div class="alert alert-block alert-info"><b>Fixing: </b> Data Types</div> 

In [2834]:
print(f"Data type [shipping_limit_date]: {order_items_df['shipping_limit_date'].dtype}")

Data type [shipping_limit_date]: object


In [2835]:
order_items_df["shipping_limit_date"] = pd.to_datetime(
    order_items_df["shipping_limit_date"]
)

In [2836]:
order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


**Insight:**\
Kolom `shipping_limit_date` telah dikonversi ke tipe data *datetime*.\
Kolom `freight_value` tetap menyertakan nilai **0.00** karena munkin saja merepresentasikan *free shipping*.

#### Membersihkan Tabel `geolocation_df`

In [2837]:
print("Duplicates in Geolocation dataset:", geolocation_df.duplicated().sum())

Duplicates in Geolocation dataset: 261831


<div class="alert alert-block alert-info"><b>Fixing: </b> Duplicates</div> 

In [2838]:
# Drop duplicates
geolocation_df = geolocation_df.drop_duplicates()
remaining_duplicates_count = geolocation_df.duplicated().sum()

In [2839]:
print("Remaining duplicates in Geolocation dataset:", remaining_duplicates_count)

Remaining duplicates in Geolocation dataset: 0


In [2840]:
geolocation_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738332 entries, 0 to 1000161
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   geolocation_zip_code_prefix  738332 non-null  int64  
 1   geolocation_lat              738332 non-null  float64
 2   geolocation_lng              738332 non-null  float64
 3   geolocation_city             738332 non-null  object 
 4   geolocation_state            738332 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 33.8+ MB


**Insight:**\
Sebanyak **261831** baris duplikat telah dihapus.

#### Membersihkan Tabel `orders_df`

<div class="alert alert-block alert-info"><b>Fixing: </b> Data Types </div> 

In [2841]:
columns = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]

for col in columns:
    print(f"Data type [{col}]: {orders_df[col].dtype}")

Data type [order_purchase_timestamp]: object
Data type [order_approved_at]: object
Data type [order_delivered_carrier_date]: object
Data type [order_delivered_customer_date]: object
Data type [order_estimated_delivery_date]: object


In [2842]:
for col in columns:
    orders_df[col] = pd.to_datetime(orders_df[col], errors="coerce")
    print(f"Data type [{col}]: {orders_df[col].dtype}")

Data type [order_purchase_timestamp]: datetime64[ns]
Data type [order_approved_at]: datetime64[ns]
Data type [order_delivered_carrier_date]: datetime64[ns]
Data type [order_delivered_customer_date]: datetime64[ns]
Data type [order_estimated_delivery_date]: datetime64[ns]


**Insight:**\
Kolom `order_purchase_timestamp`, `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, dan `order_estimated_delivery_date` telah berhasil dikonversi menjadi tipe data *datetime*.

<div class="alert alert-block alert-info"><b>Fixing: </b> Missing Values </div> 

In [2843]:
print("\nMissing values in Orders dataset:")
print(orders_df.isna().sum())


Missing values in Orders dataset:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


In [2844]:
missing_approved_at = orders_df[orders_df["order_approved_at"].isna()]
print(
    f"Number of rows with missing 'order_approved_at': {missing_approved_at.shape[0]}"
)

missing_approved_delivered = missing_approved_at[
    missing_approved_at["order_status"] == "delivered"
]

print("\nSample rows with missing 'order_approved_at' and status 'delivered':")
missing_approved_delivered.sample(3)

Number of rows with missing 'order_approved_at': 160

Sample rows with missing 'order_approved_at' and status 'delivered':


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17
72407,3c0b8706b065f9919d0505d3b3343881,d85919cb3c0529589c6fa617f5f43281,delivered,2017-02-17 15:53:27,NaT,2017-02-22 11:31:30,2017-03-03 11:47:47,2017-03-23
84999,2babbb4b15e6d2dfe95e2de765c97bce,74bebaf46603f9340e3b50c6b086f992,delivered,2017-02-18 17:15:03,NaT,2017-02-22 11:23:11,2017-03-03 18:43:43,2017-03-31


In [2845]:
# Calculate the time difference between 'order_purchase_timestamp' and 'order_approved_at' in hours
orders_df["approval_time_diff"] = (
    orders_df["order_approved_at"] - orders_df["order_purchase_timestamp"]
).dt.total_seconds() / 3600
orders_df["approval_time_diff"] = orders_df["approval_time_diff"].round(2)

# Calculate the average approval time
average_approval_time = orders_df["approval_time_diff"].mean()

# Fill missing values in 'order_approved_at' by adding the average approval time to 'order_purchase_timestamp'
orders_df["order_approved_at"] = orders_df["order_approved_at"].fillna(
    orders_df["order_purchase_timestamp"]
    + pd.to_timedelta(average_approval_time, unit="h")
)

# Round 'order_approved_at' to the nearest second
orders_df["order_approved_at"] = orders_df["order_approved_at"].dt.round("s")

orders_df.sample(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_time_diff
86895,90553773adebd97e53db3810be841c91,3f774a8ca68da8fc546c22ba0e483568,delivered,2018-02-11 11:50:35,2018-02-11 12:08:56,2018-02-14 19:40:00,2018-02-20 20:18:50,2018-03-09,0.31
42183,cd41a5bf3ffd542864ab33b17144968f,b0f4a833c13466c077bf44efca4d5b56,delivered,2017-11-24 16:33:46,2017-11-24 20:32:12,2017-12-04 16:33:32,2017-12-06 22:18:42,2017-12-15,3.97
97444,c65ca254bd0c624c11dd60d92955c989,3e4ebed68177e081dd84a416c4718ad8,delivered,2017-12-13 20:15:02,2017-12-13 20:29:21,2017-12-18 22:38:32,2017-12-22 19:27:44,2018-01-09,0.24


In [2846]:
# Check nan value in 'approval_time_diff' column
print(
    "NaN values in 'approval_time_diff':", orders_df["approval_time_diff"].isna().sum()
)

NaN values in 'approval_time_diff': 160


In [2847]:
# Fill missing values in 'approval_time_diff' with the calculated average
average_approval_time = orders_df["approval_time_diff"].mean()
orders_df["approval_time_diff"] = orders_df["approval_time_diff"].fillna(
    average_approval_time
)
orders_df["approval_time_diff"] = orders_df["approval_time_diff"].round(2)

In [2848]:
print(
    "NaN values in 'approval_time_diff':", orders_df["approval_time_diff"].isna().sum()
)

NaN values in 'approval_time_diff': 0


In [2849]:
print("\nMissing values in Orders dataset:")
orders_df.isna().sum()


Missing values in Orders dataset:


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
approval_time_diff                  0
dtype: int64

**Insight:**\
Terdapat beberapa `order_status` 'delivered' yang mempunyai nilai kosong pada `order_approved_at`.\
Kolom `order_approved_at` yang memiliki nilai kosong telah diisi dengan menghitung rata-rata waktu persetujuan dari pesanan yang sudah memiliki nilai di `order_approved_at` (selisih waktu antara `order_purchase_timestamp` dan `order_approved_at`).\

*NaT* pada kolom `order_delivered_carrier_date` dan `order_delivered_customer_date` tetap, karena disesuaikan dengan keadaan aktual dimana pesanan belum memasuki status pengiriman.

## Exploratory Data Analysis (EDA)

### Eksplorasi Data `products_df` dan `product_category_translation_df`

In [2850]:
products_df.info()

print(f"\n(rows, collumns): \t\t\t{products_df.shape}")
print(f"nunique of product_id: \t\t\t{products_df['product_id'].nunique()}")
print(
    f"nunique of product_category_name: \t{products_df['product_category_name'].nunique()}"
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32951 non-null  object 
 2   product_name_lenght         32951 non-null  float64
 3   product_description_lenght  32951 non-null  float64
 4   product_photos_qty          32951 non-null  float64
 5   product_weight_g            32951 non-null  float64
 6   product_length_cm           32951 non-null  float64
 7   product_height_cm           32951 non-null  float64
 8   product_width_cm            32951 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB

(rows, collumns): 			(32951, 9)
nunique of product_id: 			32951
nunique of product_category_name: 	74


#### Menerjemahkan `product_category_name`

In [2851]:
products_df[["product_id", "product_category_name"]].head()

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [2852]:
category_translation = product_category_translation_df.set_index(
    "product_category_name"
)["product_category_name_english"]

products_df["product_category_name"] = (
    products_df["product_category_name"]
    .map(category_translation)
    .fillna(products_df["product_category_name"])
)

products_df[["product_id", "product_category_name"]].head()

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares


In [2853]:
print("Most frequent product categories:")
products_df["product_category_name"].value_counts().head()

Most frequent product categories:


product_category_name
bed_bath_table     3029
sports_leisure     2867
furniture_decor    2657
health_beauty      2444
housewares         2335
Name: count, dtype: int64

In [2854]:
category_counts = products_df["product_category_name"].value_counts()
rarest_categories = category_counts.tail()

# Calculate percentages
total_products = category_counts.sum()
percentages = (rarest_categories / total_products * 100).round(2)

# Create a DataFrame for better formatting
output_df = pd.DataFrame({
    'Category': rarest_categories.index,
    'Count': rarest_categories.values,
    'Percentage': percentages.values
})

output_df['Percentage'] = output_df['Percentage'].map("{:.2f}%".format)

print("\nThe Rarest Product Categories")
print("-" * 60)
print(output_df.to_string(index=False))
print("-" * 60)
print(f"Total Products: {total_products}")

The rarest product category:


product_category_name
fashion_childrens_clothes    5
home_comfort_2               5
pc_gamer                     3
security_and_services        2
cds_dvds_musicals            1
Name: count, dtype: int64

In [2855]:
print(
    f"Number of products with 'unknown' category: {products_df[products_df['product_category_name'] == 'unknown'].shape[0]} \n"
)

unknown_category = products_df[products_df["product_category_name"] == "unknown"]
unknown_category[["product_id", "product_category_name"]].head()

Number of products with 'unknown' category: 610 



,product_id,product_category_name
105,a41e356c76fab66334f36de622ecbd3a,unknown
128,d8dee61c2034d6d075997acef1870e9b,unknown
145,56139431d72cd51f19eb9f7dae4d1617,unknown
154,46b48281eb6d663ced748f324108c733,unknown
197,5fb61f482620cb672f5e586bb132eae9,unknown


In [2856]:
products_df.describe().round(2)

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951.00,32951.00,32951.00,32951.00,32951.00,32951.00,32951.00
mean,48.48,771.50,2.19,2276.67,30.82,16.94,23.20
std,10.15,629.21,1.72,4281.86,16.91,13.64,12.08
min,5.00,4.00,1.00,2.00,7.00,2.00,6.00
25%,42.00,344.00,1.00,300.00,18.00,8.00,15.00
50%,51.00,604.00,1.00,700.00,25.00,13.00,20.00
75%,57.00,961.00,3.00,1900.00,38.00,21.00,30.00
max,76.00,3992.00,20.00,40425.00,105.00,105.00,118.00


**Insight:**
1. **Most frequent product categories** & **The rarest product category**
    - Kategori produk yang memiliki varian produk terbanyak adalah *bed_bath_table*.
    - Kategori produk yang memiliki varian produk paling sedikit adalah *cds_dvds_musicals*.
2. `product_name_lenght`
    - Panjang nama produk bervariasi antara **5** hingga **76** karakter dengan rata-rata panjang sebesar **48.48** karakter. 
    - Hal ini menunjukkan bahwa rata-rata nama produk relatif singkat dan deskriptif.
3. `product_description_lenght`
    - Panjang deskripsi produk berkisar dari **4** hingga **3992** karakter, dengan rata-rata deskripsi sebesar **771.5** karakter. 
    - Ini menunjukkan bahwa sebagian besar produk memiliki deskripsi yang cukup detail, namun ada beberapa produk dengan deskripsi yang sangat singkat.
4. `product_photos_qty`
    - Setiap produk memiliki antara **1** hingga **20** foto, dengan rata-rata jumlah foto sebesar **2.19**. 
    - Ini menunjukkan bahwa sebagian besar produk didokumentasikan dengan baik menggunakan setidaknya satu foto, namun ada juga yang memiliki lebih banyak foto untuk mendukung promosi produk.

### Eksplorasi Data `orders_df`, `order_items_df`, dan `order_reviews_df`

#### Eksplorasi Data `orders_df`

In [2857]:
orders_df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_time_diff
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.71
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,0.28


In [2858]:
orders_df.info()

print(f"\n(rows, collumns): \t\t{orders_df.shape}")
print(f"nunique of order_id: \t\t{orders_df['order_id'].nunique()}")
print(f"nunique of customer_id: \t{orders_df['customer_id'].nunique()}")
print(f"nunique of order_status: \t{orders_df['order_status'].nunique()}")

print(f"\nNumber of duplicate rows: \t\t{orders_df.duplicated().sum()}")
print(f"Number of duplicate in order_id: \t{orders_df['order_id'].duplicated().sum()}")
print(
    f"Number of duplicate in customer_id: \t{orders_df['customer_id'].duplicated().sum()}"
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   approval_time_diff             99441 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(3)
memory usage: 6.8+ MB

(rows, collumns): 		(99441, 9)
nunique of order_id: 		99441
nunique of customer_id:

In [2927]:
status_counts = orders_df["order_status"].value_counts()

total_orders = status_counts.sum()
percentages = (status_counts / total_orders * 100).round(2)

output_df = pd.DataFrame({
    'Order Status': status_counts.index,
    'Count': status_counts.values,
    'Percentage': percentages.values
})

output_df['Percentage'] = output_df['Percentage'].map("{:.2f}%".format)

print("\nDistribution of Order Statuses")
print("-" * 50)
print(output_df.to_string(index=False))
print("-" * 50)
print(f"Total Orders: {total_orders}")


Distribution of Order Statuses
--------------------------------------------------
Order Status  Count Percentage
   delivered  96478     97.02%
     shipped   1107      1.11%
    canceled    625      0.63%
 unavailable    609      0.61%
    invoiced    314      0.32%
  processing    301      0.30%
     created      5      0.01%
    approved      2      0.00%
--------------------------------------------------
Total Orders: 99441


##### **Memahami Distribusi `approval_time_diff`**

In [2926]:
# Assuming orders_df is your DataFrame
stats = orders_df["approval_time_diff"].describe().round(2)

# Create a DataFrame for better formatting
output_df = pd.DataFrame({
    'Statistic': stats.index,
    'Value': stats.values
})

# Format the 'Value' column to handle both integers and floats
output_df['Value'] = output_df['Value'].apply(lambda x: f"{x:.2f}" if isinstance(x, float) else f"{int(x)}")

# Print the formatted table
print("\nDescriptive Statistics of Approval Time Difference")
print("-" * 50)
print(output_df.to_string(index=False))
print("-" * 50)


Descriptive Statistics of Approval Time Difference
--------------------------------------------------
Statistic    Value
    count 99441.00
     mean    10.42
      std    26.02
      min     0.00
      25%     0.22
      50%     0.34
      75%    14.54
      max  4509.18
--------------------------------------------------


In [2861]:
delivered_status_df = orders_df[orders_df["order_status"] == "delivered"].sort_values(
    "approval_time_diff", ascending=False
)

delivered_status_df[["order_id", "order_status", "approval_time_diff"]].head()

,order_id,order_status,approval_time_diff
96251,0a93b40850d3f4becf2f276666e01340,delivered,741.44
55708,f7923db0430587601c2aef15ec4b8af4,delivered,738.45
88671,de0076b42a023f53b398ce9ab0d9009c,delivered,554.78
71651,daed0f3aefd193de33c31e21b16a3b3a,delivered,404.23
73881,9c038e10f14d12a96939a0176c4ecc99,delivered,319.53


In [2862]:
delivered_status_df[["order_id", "order_status", "approval_time_diff"]].tail()

,order_id,order_status,approval_time_diff
90080,8c006b6688741e47144a6f79f96e225d,delivered,0.0
35078,898ded7ecb1e4d92997856bd8b7a8aa1,delivered,0.0
78152,dd13617f642718bd4cdc450d197960f9,delivered,0.0
35091,2dc3f680b2788d38f7162482b575e41a,delivered,0.0
55171,380bf78bcf690a6955b8f1de575e26ba,delivered,0.0


In [2923]:
bins = [
    -float("inf"),  # Less than or equal to 0
    0,  # 0 hours
    24,  # 1 day
    48,  # 2 days
    72,  # 3 days
    96,  # 4 days
    168,  # 7 days
    336,  # 14 days
    504,  # 21 days
    720,  # 30 days
    float("inf"),  # More than 30 days
]

labels = [
    "diff <= 0",
    "0 < diff <= 24",
    "24 < diff <= 48",
    "48 < diff <= 72",
    "72 < diff <= 96",
    "96 < diff <= 168",  # 4 < diff <= 7 days
    "168 < diff <= 336",  # 7 < diff <= 14 days
    "336 < diff <= 504",  # 14 < diff <= 21 days
    "504 < diff <= 720",  # 21 < diff <= 30 days
    "diff > 720",  # > 30 days
]

# Assign time ranges to the 'approval_time_diff' column for the delivered orders DataFrame
delivered_status_df["time_range"] = pd.cut(
    delivered_status_df["approval_time_diff"], bins=bins, labels=labels, right=True
)

# Count occurrences of each time range
counts = delivered_status_df["time_range"].value_counts().reindex(labels, fill_value=0)

# Calculate the total number of delivered orders
total_delivered_orders = delivered_status_df.shape[0]

# Create a DataFrame for the output
output_df = pd.DataFrame({
    'Time Range (hours)': labels,
    'Count': counts,
    'Percentage': (counts / total_delivered_orders) * 100
})

output_df['Percentage'] = output_df['Percentage'].map("{:.3f}%".format)

print("\nDistribution of Delivered Orders by Approval Time")
print("-" * 70)
print(output_df.to_string(index=False))
print("-" * 70)
print(f"Total delivered orders: {total_delivered_orders}")


Distribution of Delivered Orders by Approval Time
----------------------------------------------------------------------
Time Range (hours)  Count Percentage
         diff <= 0   1244     1.289%
    0 < diff <= 24  78386    81.248%
   24 < diff <= 48  11872    12.305%
   48 < diff <= 72   2798     2.900%
   72 < diff <= 96   1453     1.506%
  96 < diff <= 168    657     0.681%
 168 < diff <= 336     64     0.066%
 336 < diff <= 504      1     0.001%
 504 < diff <= 720      1     0.001%
        diff > 720      2     0.002%
----------------------------------------------------------------------
Total delivered orders: 96478


In [2864]:
canceled_status_df = orders_df[orders_df["order_status"] == "canceled"].sort_values(
    "approval_time_diff", ascending=False
)

canceled_status_df[["order_id", "order_status", "approval_time_diff"]].head()

,order_id,order_status,approval_time_diff
4396,e5fa5a7210941f7d56d0208e4e071d35,canceled,781.03
53475,490291524fddde2b31c2e6bec3d9e6da,canceled,676.07
10071,809a282bbd5dbcabb6f2f724fca862ec,canceled,573.87
40076,7fd4b0e047195ca197c3660772a8d8c0,canceled,248.28
62406,123e27a1a4d0b2481d8618ac3dff7d4e,canceled,242.58


In [2865]:
canceled_status_df[["order_id", "order_status", "approval_time_diff"]].tail()

,order_id,order_status,approval_time_diff
71603,b17fd4c033b06e8d888de8ea6105ef9d,canceled,0.0
67963,5b9680f27b5067afded00b23f9cb4d61,canceled,0.0
28846,9abe82df39e950e1e0c1a5969e22571e,canceled,0.0
28455,d01c5b46e00bd214519fe9f64bbb2649,canceled,0.0
80797,983e16bfbe23e60f23a6fcb9c4792204,canceled,0.0


In [2925]:
canceled_status_df["time_range"] = pd.cut(
    canceled_status_df["approval_time_diff"], bins=bins, labels=labels, right=True
)

counts = canceled_status_df["time_range"].value_counts().reindex(labels, fill_value=0)

total_canceled_orders = canceled_status_df.shape[0]

output_df = pd.DataFrame({
    'Time Range': labels,
    'Count': counts,
    'Percentage': (counts / total_canceled_orders) * 100
})

output_df['Percentage'] = output_df['Percentage'].map("{:.2f}%".format)

print("\nDistribution of Canceled Orders by Approval Time")
print("-" * 60)
print(output_df.to_string(index=False))
print("-" * 60)
print(f"Total canceled orders: {total_canceled_orders}")


Distribution of Canceled Orders by Approval Time
------------------------------------------------------------
       Time Range  Count Percentage
        diff <= 0     18      2.88%
   0 < diff <= 24    525     84.00%
  24 < diff <= 48     56      8.96%
  48 < diff <= 72     10      1.60%
  72 < diff <= 96      8      1.28%
 96 < diff <= 168      1      0.16%
168 < diff <= 336      4      0.64%
336 < diff <= 504      0      0.00%
504 < diff <= 720      2      0.32%
       diff > 720      1      0.16%
------------------------------------------------------------
Total canceled orders: 625


**Insight:**
1. **Pesanan yang dibatalkan cenderung disetujui lebih cepat**
    - Berdasarkan `approval_time_diff`:
        - `canceled`: **84.00%** disetujui dalam 24 jam pertama
        - `delivered`: **81.25%** disetujui dalam 24 jam pertama
    - Terindikasi bahwa **kecepatan persetujuan bukan faktor utama pembatalan pesanan oleh pelanggan.**
2. Data ini menunjukkan bahwa pembeli tidak membatalkan pesanan karena terlalu lama persetujuan. Sebaliknya, pembatalan tampaknya lebih dipengaruhi oleh faktor-faktor lain yang tidak terlihat dalam data ini.

##### **Drop Unnecessary Data**

In [2867]:
value_counts = orders_df["order_status"].value_counts()
percentage = (value_counts / value_counts.sum()) * 100

percentage_df = pd.DataFrame(
    {"counts": value_counts, "percentage": percentage.round(3)}
)

percentage_df

,counts,percentage
order_status,,
delivered,96478,97.020
shipped,1107,1.113
canceled,625,0.629
unavailable,609,0.612
invoiced,314,0.316
processing,301,0.303
created,5,0.005
approved,2,0.002


#### Eksplorasi Data `orders_df` dan `order_reviews_df`

In [2868]:
merged_df = pd.merge(
    orders_df, order_reviews_df, on="order_id", how="left", suffixes=("", "_reviews")
)

merged_df["review_score"] = merged_df["review_score"].astype("Int64")

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99992 entries, 0 to 99991
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99992 non-null  object        
 1   customer_id                    99992 non-null  object        
 2   order_status                   99992 non-null  object        
 3   order_purchase_timestamp       99992 non-null  datetime64[ns]
 4   order_approved_at              99992 non-null  datetime64[ns]
 5   order_delivered_carrier_date   98199 non-null  datetime64[ns]
 6   order_delivered_customer_date  97005 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99992 non-null  datetime64[ns]
 8   approval_time_diff             99992 non-null  float64       
 9   review_id                      99224 non-null  object        
 10  review_score                   99224 non-null  Int64         
 11  review_comment_

In [2902]:
merged_df.head().T

,0,1,2,3,4
order_id,e481f51cbdc54678b7cc49136f2d6af7,53cdb2fc8bc7dce0b6741e2150273451,47770eb9100c2d0c44946d9cf07ec65d,949d5b44dbf5de918fe9c16f97b45f8a,ad21c59c0840e6cb83a9ceb5573f8159
customer_id,9ef432eb6251297304e76186b10a928d,b0830fb4747a6c6d20dea0b8c802d7ef,41ce2a54c0b03bf3443c3d931a367089,f88197465ea7920adcdbec7375364d82,8ab97904e6daea8866dbdbc4fb7aad2c
order_status,delivered,delivered,delivered,delivered,delivered
order_purchase_timestamp,2017-10-02 10:56:33,2018-07-24 20:41:37,2018-08-08 08:38:49,2017-11-18 19:28:06,2018-02-13 21:18:39
order_approved_at,2017-10-02 11:07:15,2018-07-26 03:24:27,2018-08-08 08:55:23,2017-11-18 19:45:59,2018-02-13 22:20:29
order_delivered_carrier_date,2017-10-04 19:55:00,2018-07-26 14:31:00,2018-08-08 13:50:00,2017-11-22 13:39:59,2018-02-14 19:46:34
order_delivered_customer_date,2017-10-10 21:25:13,2018-08-07 15:27:45,2018-08-17 18:06:29,2017-12-02 00:28:42,2018-02-16 18:17:02
order_estimated_delivery_date,2017-10-18 00:00:00,2018-08-13 00:00:00,2018-09-04 00:00:00,2017-12-15 00:00:00,2018-02-26 00:00:00
approval_time_diff,0.18,30.71,0.28,0.3,1.03
review_id,a54f0611adc9ed256b57ede6b6eb5114,8d5266042046a06655c8db133d120ba5,e73b67b67587f7644d5bd1a52deb1b01,359d03e676b3c069f62cadba8dd3f6e8,e50934924e227544ba8246aeb3770dd4


In [2917]:
output = merged_df["review_score"].value_counts().sort_index()

df = pd.DataFrame({"Review Score": output.index, "Count": output.values})
df["Percentage"] = df["Count"] / df["Count"].sum() * 100

df.set_index("Review Score", inplace=True)

df["Percentage"] = df["Percentage"].map("{:.2f}%".format)

# Print the formatted table
print("\nDistribution of Score Review")
print("-" * 40)
print(df.to_string())
print("-" * 40)
print(f"Total: {df['Count'].sum()}")


Distribution of Score Review
----------------------------------------
              Count Percentage
Review Score                  
1             11424     11.51%
2              3151      3.18%
3              8179      8.24%
4             19142     19.29%
5             57328     57.78%
----------------------------------------
Total: 99224


In [2870]:
merged_df.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_carrier_date     1793
order_delivered_customer_date    2987
order_estimated_delivery_date       0
approval_time_diff                  0
review_id                         768
review_score                      768
review_comment_title              768
review_comment_message            768
review_creation_date              768
review_answer_timestamp           768
dtype: int64

In [2912]:
print("NaN values in review_score:")
merged_df[merged_df["review_score"].isna()].head(3)

NaN values in review_score:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,approval_time_diff,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
16,403b97836b0c04a622354cf531062e5f,738b086814c6fcc74b8cc583f8516ee3,delivered,2018-01-02 19:00:43,2018-01-02 19:09:04,2018-01-03 18:19:09,2018-01-20 01:38:59,2018-02-06,0.14,NaN,<NA>,NaN,NaN,NaT,NaT
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaT,2018-02-07,14.99,NaN,<NA>,NaN,NaN,NaT,NaT
311,4906eeadde5f70b308c20c4a8f20be02,4e7656e34357b93f14b40c6400ca3f6e,delivered,2017-12-08 04:45:26,2017-12-12 03:50:30,2017-12-12 17:43:21,2018-01-09 18:04:58,2018-01-03,95.08,NaN,<NA>,NaN,NaN,NaT,NaT


In [2906]:
# Show rows that have canceled status and have a value in the review_comment_message column that is not 'empty'
canceled_reviews = merged_df[
    (merged_df["order_status"] == "canceled")
    & (merged_df["review_comment_message"] != "empty")
]

# Filter the desired rows and columns
filtered_df = canceled_reviews[
    [
        "order_id",
        "order_status",
        "customer_id",
        "review_id",
        "review_score",
        "review_comment_message",
    ]
].sample(5)

filtered_df.T

,99041,47957,24890,25260,91113
order_id,2be6c8b958e16856397259974d4abee9,24f07a52ca6b749c618b46ddb2e94b8a,a9957adf297a6be5e8625b3e93329c6d,2e8a59579f027832343436a97c22499b,cf5508a6ae55421646b0683bb605ac74
order_status,canceled,canceled,canceled,canceled,canceled
customer_id,ccbce7bcb13ee88adcf78890cae0ea42,2fead6d196da5adf133e4ddb08f51c8c,7e779aeb9722a44ac85680883c9ee20f,4dcdb96c4c889577e5b04124b6ce5cff,e42274a986c58b500513a7d57a4fed2d
review_id,60a43f2ae6645ae3c49a96bac275680b,bf3adf61d4e6dfb0ca4e3b957630b8bb,aed16082f275e17a2d85693d0f971d0e,b48d676f6cbb5bfe53f834197db3dfe4,5c76615cb4d96fe9ddfb9121c7594436
review_score,1,1,1,1,1
review_comment_message,"Meu produto não foi entregue, estava previsto para 19/04 e até agora nada. Já passou do prazo de entrega.Pois já veio até cobrando a primeira parcela no meu cartão.\r\nJá mandei email e o site não respo","Estou aguardando meu produto ,dei numero errado mais ja dei o certo e ate agora nao recebi o produto","Não recebi meu pedido, e não é a primeira vez que ocorre nesse site.","Produto não entregue dentro do prazo, e falta de posicionamento do produto. Onde o vendedor não soube me dizer o que houve com o produto.\r\n",Não recebi o produto até o momento.


In [2909]:
# Calculate the length of each review message
canceled_reviews.loc[:, "review_length"] = canceled_reviews[
    "review_comment_message"
].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

# Sort by review length (descending) and review score (ascending)
canceled_reviews = canceled_reviews.sort_values(
    by=["review_length", "review_score"], ascending=[False, True]
)

# Select the top 5 rows based on the sorted values
selected_reviews = canceled_reviews[
    [
        "order_id",
        "order_status",
        "customer_id",
        "review_id",
        "review_score",
        "review_comment_message",
    ]
].head(5)

selected_reviews.T

,74781,24968,6680,20033,40197
order_id,d7cc5fcfee9af53fcd78bb47e4e55584,642a8a89c36a8b2f4884cd8e68168247,5b4858f2dcf69731f2285a38829bfa7c,ce0a53468ddde09e79f73edba370ceec,f3bd213a969549dd53ed48872f4bb69d
order_status,canceled,canceled,canceled,canceled,canceled
customer_id,6c081dfe77db2d63f336f33043f0036a,64fc1ea0b3ef7d792474f9cce8ddbfb1,c8223f76047e09bea798462f2f46ffa9,be661c8beed58e228518d223a539700e,1b4970ad8aaf073fd9164c55384584a5
review_id,13fda443af3ce6df563ffe6956042056,fb83f879a7eff7a6bdc9c9860bc72615,b439df4e47d50f3d06b6395d9b3867c2,0f7d99e8dd13bd0928d27904e11b0d19,b943aa9b8877835e183d136d2712479e
review_score,1,1,1,1,1
review_comment_message,"Gostaria de saber através da lannister.com,como vai ficar a minha situação,já que a targaryeni esta vinculada a americana.com. Realizei a compra virtual através de uma vendedora das Lojas lannister S/A.","nao recomendo esta loja, como nunca tinha tido problema em comprar pelo baratheon n atentei a pesquisar sobre o fornecedor,depois do prazo da entrega ter passado fui pesquisar e existe varias reclamçao","0 correio não entregou a minha encomenda por que meu nome estava incompleto, por favor mande o meu produto ou devolva meu dinheiro, fui no correio varias vezes mas não quiseram entregar o meu produto.","A loja foi irresponsável, pois não emitiu a nota fiscal e nem avisou se tinha o produto ou quanto tempo iria ter, ficou apenas dizendo que estava aguardando a emissão da nota fiscal, isso levou um mês","10 dias após efetuar a compra, gerar o boleto e pagar, recebo um e-mail cancelando a compra por falta do produto.\r\nPara o estorno do valor me obrigaram a ir em uma agência bancaria e retirar o valor!"


In [2911]:
output = canceled_reviews["review_score"].value_counts().sort_index()

df = pd.DataFrame({"Review Score": output.index, "Count": output.values})

df.set_index("Review Score", inplace=True)

print("\nCanceled Reviews by Score")
print("-" * 25)
print(df.to_string())
print("-" * 25)
print(f"Total: {df['Count'].sum()}")


Canceled Reviews by Score
-------------------------
              Count
Review Score       
1               311
2                35
3                26
4                 6
5                28
-------------------------
Total: 406


In [2896]:
# Select the first 5 rows and store in a new DataFrame
head_canceled_reviews_df = canceled_reviews.head(5).copy()

# Translate the review messages directly within the new DataFrame
head_canceled_reviews_df["Review Message"] = head_canceled_reviews_df[
    "review_comment_message"
].apply(
    lambda x: (
        GoogleTranslator(source="auto", target="id").translate(x) if pd.notna(x) else ""
    )
)

translated_df = head_canceled_reviews_df[
    [
        "order_id",
        "order_status",
        "customer_id",
        "review_id",
        "review_score",
        "Review Message",
    ]
].rename(
    columns={
        "order_id": "Order ID",
        "order_status": "Order Status",
        "customer_id": "Customer ID",
        "review_id": "Review ID",
        "review_score": "Review Score",
    }
)

pd.set_option("display.max_colwidth", None)

translated_df.T

,74781,24968,6680,20033,40197
Order ID,d7cc5fcfee9af53fcd78bb47e4e55584,642a8a89c36a8b2f4884cd8e68168247,5b4858f2dcf69731f2285a38829bfa7c,ce0a53468ddde09e79f73edba370ceec,f3bd213a969549dd53ed48872f4bb69d
Order Status,canceled,canceled,canceled,canceled,canceled
Customer ID,6c081dfe77db2d63f336f33043f0036a,64fc1ea0b3ef7d792474f9cce8ddbfb1,c8223f76047e09bea798462f2f46ffa9,be661c8beed58e228518d223a539700e,1b4970ad8aaf073fd9164c55384584a5
Review ID,13fda443af3ce6df563ffe6956042056,fb83f879a7eff7a6bdc9c9860bc72615,b439df4e47d50f3d06b6395d9b3867c2,0f7d99e8dd13bd0928d27904e11b0d19,b943aa9b8877835e183d136d2712479e
Review Score,1,1,1,1,1
Review Message,"Saya ingin tahu melalui lannister.com seperti apa situasi saya nanti, karena targaryeni terhubung ke americana.com. Saya melakukan pembelian virtual melalui penjual di Lojas Lannister S/A.","Saya tidak merekomendasikan toko ini, karena saya tidak pernah mengalami masalah saat membeli melalui Baratheon, saya tidak mencoba meneliti pemasoknya, setelah batas waktu pengiriman lewat saya memeriksanya dan ada beberapa keluhan.","Kurir tidak mengirimkan pesanan saya karena nama saya tidak lengkap, silakan kirim produk saya atau kembalikan uang saya, saya pergi ke kantor pos beberapa kali tetapi mereka tidak mau mengirimkan produk saya.","Toko tidak bertanggung jawab karena tidak mengeluarkan invoice atau menginformasikan apakah ada produk atau berapa lama, hanya dikatakan menunggu invoice dikeluarkan, butuh waktu sebulan","10 hari setelah melakukan pembelian, membuat faktur dan membayar, saya menerima email yang membatalkan pembelian karena kekurangan produk.\r\nUntuk mengembalikan jumlah tersebut, mereka memaksa saya pergi ke cabang bank dan menarik jumlah tersebut!"


In [2898]:
# Filter and sort reviews with a score of 5 by review length in descending order
top_5_5star_reviews = (
    canceled_reviews.loc[canceled_reviews["review_score"] == 5]
    .nlargest(5, "review_length")
    .copy()
)

# Translate the review messages directly within the new DataFrame
top_5_5star_reviews["Review Message"] = top_5_5star_reviews[
    "review_comment_message"
].apply(
    lambda x: (
        GoogleTranslator(source="auto", target="id").translate(x) if pd.notna(x) else ""
    )
)

translated_fivestar_df = top_5_5star_reviews[
    [
        "order_id",
        "order_status",
        "customer_id",
        "review_id",
        "review_score",
        "Review Message",
    ]
].rename(
    columns={
        "order_id": "Order ID",
        "order_status": "Order Status",
        "customer_id": "Customer ID",
        "review_id": "Review ID",
        "review_score": "Review Score",
    }
)

translated_fivestar_df.T

,8590,93618,65854,25790,58410
Order ID,7142b298b22a79a285d2ce792dc533a2,7a56ded9b696cca5e892f56f49b1921a,ded456690bcb2ffd4def86541479037f,1857e77a4bad8abb2db929f107430663,cadd4c5765abf658117f2cc7e2a639c2
Order Status,canceled,canceled,canceled,canceled,canceled
Customer ID,d2fac3a405031b2adb1739b20cae646a,caca87282df2b6fce5e717b93e548b92,e72188520e7450d41369c84fed0a4b68,616833eaef04d25b67207119cd6936dc,939c6ecf27d8c32e7956b4a2a468326d
Review ID,0624d0ec20d86b39b8f4936da80afff7,c8cbb7509c516719ed41a11bf31d757f,a38a071b913a47b0990806e07409d454,cab94b2b92fe84fc842095bdf1451904,0c9b6ca9b3beaf0fd5d621f63844ea01
Review Score,5,5,5,5,5
Review Message,Mereka tidak memberikan kepuasan sedikit pun kepada pelanggan. Pengiriman dijadwalkan pada 13/03/17 dan itu adalah satu-satunya komunikasi yang saya miliki. Saya mengirim email dan tidak dijawab pada 10/03. Pesanan akan dibatalkan,"Ada masalah dengan pengiriman, kantor pos tidak mengirimkannya ke orang di agen meskipun dengan kode pelacakan dan dokumen. Mereka menginformasikan bahwa produk tersebut tidak ada di unit.","Saya menerima email yang memberitahukan tentang pengembalian produk, namun saya tidak mengetahui permintaan ini, bukan saya yang membuatnya. Saya masih menunggu pengiriman dalam batas waktu yang ditentukan.","Produk dikirim seperti yang diiklankan, termasuk sambungan dan katup. Bahkan gulungan pita penyegel benang disertakan dengan sambungannya. Saya sangat merekomendasikannya.","Saya berharap untuk menerimanya minggu ini, yang tidak akan terlalu berlebihan, namun... penundaan lebih dari itu akan membuat saya menyerah pada baratheon."


**Insight:**\

#### Eksplorasi Data `order_items_df`

In [ ]:
print(order_items_df.info())

print(f"\nnunique of order_id: \t\t{order_items_df['order_id'].nunique()}")
print(f"nunique of product_id: \t\t{order_items_df['product_id'].nunique()}")
print(f"nunique of seller_id: \t\t{order_items_df['seller_id'].nunique()}")

print(f"\nNumber of duplicate rows: \t{order_items_df.duplicated().sum()}")

In [ ]:
merged_items_reviews_df = pd.merge(
    order_reviews_df, order_items_df, on="order_id", how="left"
)

selected_colls = merged_items_reviews_df[
    [
        "order_id",
        "review_id",
        "review_score",
        "order_item_id",
        "seller_id",
        "product_id",
    ]
]

multi_item_orders = selected_colls.groupby("order_id").filter(
    lambda x: x["product_id"].nunique() > 1
)

"Sample of multi-item orders:\n", multi_item_orders.head(6)

**Insight:**
- `multi_item_order`:
    - `order_id` <span style="color:orange">b18dcdf73be66366873cd26c5724d1dc</span> memiliki beberapa item yang berbeda (`order_item_id` 1, 2, 3, dan 4). Semua item ini terkait dengan satu ulasan (berdasarkan `review_id`), dan pelanggan memberikan skor **1** untuk keseluruhan pesanan.
    - `order_id` <span style="color:orange">d7bd0e4afdf94846eb73642b4e3e75c3</span> juga memuat lebih dari satu item, dan pelanggan memberikan skor **3**.
- Ini memberikan gambaran jelas bahwa dalam beberapa kasus, satu `order_id` memang berisi beberapa item, dan setiap item mungkin memiliki detail produk yang berbeda. Namun, ulasan diberikan satu kali untuk seluruh pesanan (berdasarkan `order_id`), meskipun pesanan tersebut terdiri dari beberapa produk (berdasarkan `order_item_id`).

#### Eksplorasi Data `merged_items_reviews_df`

In [ ]:
merged_items_reviews_df.head()

In [ ]:
merged_items_reviews_df.info()

print(f"\n(rows, collumns): \t\t\t{merged_items_reviews_df.shape}")
print(f"\nNumber of duplicate rows: \t\t{merged_items_reviews_df.duplicated().sum()}")
print(
    f"Number of duplicate in order_id: \t{merged_items_reviews_df['order_id'].duplicated().sum()}"
)
print(
    f"Number of duplicate in review_id: \t{merged_items_reviews_df['review_id'].duplicated().sum()}"
)
print(
    f"Number of duplicate in product_id: \t{merged_items_reviews_df['product_id'].duplicated().sum()}"
)

print("\nnunique of order_id: \t", merged_items_reviews_df["order_id"].nunique())
print("nunique of review_id: \t", merged_items_reviews_df["review_id"].nunique())

In [ ]:
merged_items_reviews_df = pd.merge(
    merged_items_reviews_df,
    products_df[["product_id", "product_category_name"]],
    on="product_id",
    how="left",
)

cols = list(merged_items_reviews_df.columns)
product_id_index = cols.index("product_id")
cols.insert(product_id_index + 1, cols.pop(cols.index("product_category_name")))
merged_items_reviews_df = merged_items_reviews_df[cols]

print(merged_items_reviews_df.head())

#### Eksplorasi Data `merged_orders_items_reviews_df`

In [ ]:
merged_df = pd.merge(merged_items_reviews_df, orders_df, on="order_id", how="left")
merged_df.head()

In [ ]:
print(merged_df.isna().sum())

In [ ]:
# cari order id yang memiliki review_score dengan status canceled
canceled_orders = merged_df[merged_df["order_status"] == "canceled"]

# cek baris data order_id yang memiliki review_score dengan status canceled
print(canceled_orders[["order_id", "review_score", "order_status"]].head())

In [ ]:
# tampilkan row order id 8d4c637f1accf7a88a4555f02741e606
pd.set_option("display.max_columns", None)

print(
    canceled_orders[
        canceled_orders["order_id"].isin(
            ["8d4c637f1accf7a88a4555f02741e606", "96b947c986b42f213c9e6b8ccccedb89"]
        )
    ]
)

### Eksplorasi Data `order_payments_df`

In [ ]:
print(order_payments_df.head())

print(f"\n(rows, collumns): \t\t\t{order_payments_df.shape}")
print(f"nunique of order_id: \t\t\t{order_payments_df['order_id'].nunique()}")
print(f"nunique of payment_type: \t\t{order_payments_df['payment_type'].nunique()}")

print(f"\nNumber of duplicate rows: \t\t{order_payments_df.duplicated().sum()}")
print(
    f"Number of duplicate in order_id: \t{order_payments_df['order_id'].duplicated().sum()}"
)

In [ ]:
# Baris yang duplikat berdasarkan order_id
duplicated_order_id = order_payments_df[
    order_payments_df.duplicated("order_id", keep=False)
]
print(duplicated_order_id.sort_values(by="order_id", ascending=False).head(4))

In [ ]:
print(
    order_payments_df.pivot_table(
        index="payment_type", values="order_id", aggfunc="nunique"
    ).sort_values(by="order_id", ascending=False)
)

In [ ]:
# Rows yang memiliki nilai not_defined pada kolom payment_type
not_defined_payment = order_payments_df[
    order_payments_df["payment_type"] == "not_defined"
]
print(not_defined_payment)

In [ ]:
print(order_payments_df.describe().round(2))

**Insight:**
- `duplicated_order_id`
    - Dataset ini mencakup **103886** entri, dengan **99440** pesanan unik. 
    - Terdapat **4446** pesanan yang memiliki entri duplikat pada `order_id`
    - Ini menunjukkan bahwa beberapa pesanan memiliki lebih dari satu pembayaran, juga menunjukkan pesanan dibayar dengan beberapa metode atau cicilan.
- 

### Eksplore Data `sellers_df`

In [ ]:
print(sellers_df.head())

print(f"\n(rows, collumns): \t\t\t{sellers_df.shape}")
print(f"nunique of seller_id: \t\t\t{sellers_df['seller_id'].nunique()}")
print(
    f"nunique of seller_zip_code_prefix: \t{sellers_df['seller_zip_code_prefix'].nunique()}"
)
print(f"nunique of seller_city: \t\t{sellers_df['seller_city'].nunique()}")
print(f"nunique of seller_state: \t\t{sellers_df['seller_state'].nunique()}")

print(f"\nNumber of duplicate rows: \t\t{sellers_df.duplicated().sum()}")
print(
    f"Number of duplicate in seller_id: \t{sellers_df['seller_id'].duplicated().sum()}"
)

In [ ]:
print(
    sellers_df.pivot_table(
        index="seller_state", values="seller_id", aggfunc="count"
    ).sort_values(by="seller_id", ascending=False)
)

### Eksplore Data `customers_df`

In [ ]:
print(customers_df.head())

print(f"\n(rows, collumns): \t\t\t{customers_df.shape}")
print(f"nunique of customer_id: \t\t{customers_df['customer_id'].nunique()}")
print(
    f"nunique of customer_unique_id: \t\t{customers_df['customer_unique_id'].nunique()}"
)

print(f"\nNumber of duplicate rows: \t\t\t{customers_df.duplicated().sum()}")
print(
    f"Number of duplicate in customer_id: \t\t{customers_df['customer_id'].duplicated().sum()}"
)
print(
    f"Number of duplicate in customer_unique_id: \t{customers_df['customer_unique_id'].duplicated().sum()}"
)

In [ ]:
# Baris yang duplikat berdasarkan customer_unique_id
duplicated_customer_unique_id = customers_df[
    customers_df.duplicated("customer_unique_id", keep=False)
]
print(
    duplicated_customer_unique_id.sort_values(
        by="customer_unique_id", ascending=False
    ).head()
)

### Eksplorasi Data `geolocation_df`

In [ ]:
print(geolocation_df.head())

print(f"\n(rows, collumns): \t\t\t{geolocation_df.shape}")
print(
    f"nunique of geolocation_zip_code_prefix: {geolocation_df['geolocation_zip_code_prefix'].nunique()}"
)
print(f"nunique of geolocation_lat: \t\t{geolocation_df['geolocation_lat'].nunique()}")
print(f"nunique of geolocation_lng: \t\t{geolocation_df['geolocation_lng'].nunique()}")

print(f"\nNumber of duplicate rows: \t\t\t\t{geolocation_df.duplicated().sum()}")
print(
    f"Number of duplicate in geolocation_zip_code_prefix: \t{geolocation_df['geolocation_zip_code_prefix'].duplicated().sum()}"
)

In [ ]:
# Baris yang duplikat berdasarkan geolocation_zip_code_prefix
duplicated_geolocation_zip_code_prefix = geolocation_df[
    geolocation_df.duplicated("geolocation_zip_code_prefix", keep=False)
]
print(
    duplicated_geolocation_zip_code_prefix.sort_values(
        by="geolocation_zip_code_prefix", ascending=False
    ).head()
)

Setiap titik geolokasi mungkin mewakili lokasi yang berbeda dalam kode pos yang sama dalam kota yang sama.

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2